In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Lasso
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from scipy.stats import norm, skew, ttest_ind, f_oneway
from sklearn.preprocessing import LabelEncoder, StandardScaler, PowerTransformer

from sklearn.neighbors import KNeighborsClassifier

In [3]:
from google.colab import files
uploaded = files.upload()

Saving penguins_size(1).csv to penguins_size(1).csv


In [4]:
df= pd.read_csv('penguins_size(1).csv')
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [5]:
df.shape

(344, 7)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   culmen_length_mm   342 non-null    float64
 3   culmen_depth_mm    342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                334 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [7]:
df.describe()

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


In [8]:
df.isnull().sum()

,0
species,0
island,0
culmen_length_mm,2
culmen_depth_mm,2
flipper_length_mm,2
body_mass_g,2
sex,10


In [9]:
df.dropna(inplace=True)

In [10]:
print(df.isnull().sum())

species              0
island               0
culmen_length_mm     0
culmen_depth_mm      0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64



### **DATA PREROCESSING**

LABEL ENCODER

In [11]:
label_encoders = {}
categorical_columns = [
    'sex', 'island', 'species'
]

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
numerical_columns =  [ 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g']

REMOVING OUTLIERS

In [12]:
def remove_outliers(df,columns ):       # Outlier detection of the Numeric columns
  for col in columns:

    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

  return df

df = remove_outliers(df , numerical_columns)

NORMALIZE NUMERIC FEATURES

In [13]:
def normalize(df,columns):
  pt=PowerTransformer(method='yeo-johnson')
  for col in columns:
    skewness=skew(df[col])
    if abs(skewness)>0.5:
        df[col]=pt.fit_transform(df[col].values.reshape(-1,1))
    return df

data = normalize(df,numerical_columns)

STANDARD SCALER IN NUMERICAL FEATURES

In [14]:
# Scale numerical features
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [15]:
x =df.drop('species',axis=1)
y =df['species']

In [16]:
clf1= LogisticRegression()
clf2=RandomForestClassifier()
clf3=KNeighborsClassifier()

In [17]:
estimators = [('lr',clf1),('rf',clf2),('knn',clf3)]

In [18]:
x.shape


(334, 6)

In [19]:
y.shape

(334,)

In [20]:
for estimator in estimators:
    pop = cross_val_score(estimator[1],x,y,cv=10,scoring='accuracy')
    print(estimator[0],np.round(np.mean(pop), 2))

lr 0.99
rf 0.99
knn 0.99


In [21]:
from sklearn.ensemble import VotingClassifier

### **HARD VOTING**

In [22]:
vc=VotingClassifier(estimators=estimators)
do = cross_val_score(vc,x,y,cv=10,scoring='accuracy')
print(np.round(np.mean(do),2))

1.0


### **SOFT VOTING**

In [23]:
vc1=VotingClassifier(estimators=estimators,voting='soft')
score = cross_val_score(vc1,x,y,cv=10,scoring='accuracy')
print(np.round(np.mean(score),2))

1.0


### **WEGHTED VOTING**

in this voting we assign a cetain number of weights to our model

In [24]:
for i in range(1,4):
  for j in range(1,4):
      for k in range(1,4):
          vc2=VotingClassifier(estimators=estimators,voting='soft',weights=[i,j,k])
          sew = cross_val_score(vc2,x,y,cv=10,scoring='accuracy')
          print('for i={},j={},k={}'.format(i,j,k),np.round(np.mean(sew),2))

for i=1,j=1,k=1 1.0
for i=1,j=1,k=2 1.0
for i=1,j=1,k=3 1.0
for i=1,j=2,k=1 1.0
for i=1,j=2,k=2 1.0
for i=1,j=2,k=3 1.0
for i=1,j=3,k=1 1.0
for i=1,j=3,k=2 1.0
for i=1,j=3,k=3 1.0
for i=2,j=1,k=1 1.0
for i=2,j=1,k=2 1.0
for i=2,j=1,k=3 1.0
for i=2,j=2,k=1 1.0
for i=2,j=2,k=2 1.0
for i=2,j=2,k=3 1.0
for i=2,j=3,k=1 1.0
for i=2,j=3,k=2 1.0
for i=2,j=3,k=3 1.0
for i=3,j=1,k=1 1.0
for i=3,j=1,k=2 1.0
for i=3,j=1,k=3 1.0
for i=3,j=2,k=1 1.0
for i=3,j=2,k=2 1.0
for i=3,j=2,k=3 1.0
for i=3,j=3,k=1 1.0
for i=3,j=3,k=2 1.0
for i=3,j=3,k=3 1.0


In [26]:
from sklearn.svm import SVC

In [27]:
from sklearn.datasets import make_classification

x,y=make_classification(n_samples=1000,n_features=20,n_informative=15,n_redundant=5,random_state=2)

svn1 = SVC(probability=True,kernel='poly',degree=1)
svn2 = SVC(probability=True,kernel='poly',degree=2)
svn3 = SVC(probability=True,kernel='poly',degree=3)
svn4 = SVC(probability=True,kernel='poly',degree=4)
svn5 = SVC(probability=True,kernel='poly',degree=5)

estimators = [('svn1',svn1),('svn2',svn2),('svn3',svn3),('svn4',svn4),('svn5',svn5)]

for estimator in estimators:
    scores = cross_val_score(estimator[1],x,y,cv=10,scoring='accuracy')
    print(estimator[0],np.round(np.mean(scores), 2))

svn1 0.85
svn2 0.85
svn3 0.89
svn4 0.81
svn5 0.86


### **HARD VOTING**

In [28]:
vc=VotingClassifier(estimators=estimators)
doi = cross_val_score(vc,x,y,cv=10,scoring='accuracy')
print(np.round(np.mean(doi),2))

0.91


### **SOFT VOTING**

In [29]:
vc1=VotingClassifier(estimators=estimators,voting='soft')
scoreH = cross_val_score(vc1,x,y,cv=10,scoring='accuracy')
print(np.round(np.mean(scoreH),2))

0.93


### **WEIGHTED VOTING**

In [37]:
for i in range(1,3):
  for j in range(1,3):
    for k in range(1,3):
      for l in range (1,2):
        for m in range (2,4):
            vc2=VotingClassifier(estimators=estimators,weights=[i,j,k,l,m])
            sew = cross_val_score(vc2,x,y,cv=10,scoring='accuracy')
            print(f'for i={i},j={j},k={k},l={l},m={m}'.format(i,j,k,l,m),np.round(np.mean(sew),2))

for i=1,j=1,k=1,l=1,m=2 0.88
for i=1,j=1,k=1,l=1,m=3 0.88
for i=1,j=1,k=2,l=1,m=2 0.9
for i=1,j=1,k=2,l=1,m=3 0.88
for i=1,j=2,k=1,l=1,m=2 0.91
for i=1,j=2,k=1,l=1,m=3 0.88
for i=1,j=2,k=2,l=1,m=2 0.91
for i=1,j=2,k=2,l=1,m=3 0.9
for i=2,j=1,k=1,l=1,m=2 0.91
for i=2,j=1,k=1,l=1,m=3 0.88
for i=2,j=1,k=2,l=1,m=2 0.9
for i=2,j=1,k=2,l=1,m=3 0.89
for i=2,j=2,k=1,l=1,m=2 0.91
for i=2,j=2,k=1,l=1,m=3 0.91
for i=2,j=2,k=2,l=1,m=2 0.91
for i=2,j=2,k=2,l=1,m=3 0.9
